# Andalytics y Eibisi Corp. en busca del talento perdido

## Bonus: Base de datos en SQL

1. Conexión al servidor de MySQL
2. Creación de las tablas
3. Inserción de los datos en las tablas



## 1. Conexión al servidor

In [ ]:
# Importamos las librerías necesarias
import mysql.connector
from mysql.connector import errorcode
import pandas as pd      

# Nos conectamos al servidor MySQL
cnx = mysql.connector.connect(
    user='root', 
    password='AlumnaAdalab',
    host='localhost')

# Creamos el cursor
mycursor = cnx.cursor()

# Creamos el esquema 
mycursor.execute("CREATE DATABASE IF NOT EXISTS EibisiCorp")

# Confirmamos los cambios
cnx.commit()

# Seleccionamos la base de datos para trabajar
mycursor.execute("USE EibisiCorp")


## 2. Creación de las tablas

In [ ]:
# Creamos una tabla con los datos personales de los empleados
mycursor.execute('''CREATE TABLE IF NOT EXISTS personal_info (
                id_employee INT AUTO_INCREMENT PRIMARY KEY,
                employee_number VARCHAR(10),
                name VARCHAR(30),
                age INT,
                gender VARCHAR(10),
                marital_status VARCHAR(10),
                distance_from_home INT,
                date_birth INT,
                remote_work VARCHAR(10),
                education INT,
                education_field VARCHAR(50))''')

#Tabla con los datos de la estructura laboral de la empresa
mycursor.execute('''CREATE TABLE IF NOT EXISTS employment_info (
                id_job INT AUTO_INCREMENT PRIMARY KEY,
                id_employee INT,
                department VARCHAR(50),
                job_role VARCHAR(50),
                job_level INT,
                business_travel VARCHAR(20),
                overtime VARCHAR(10),
                FOREIGN KEY (id_employee) REFERENCES personal_info(id_employee))''')

# Tabla de niveles de satisfacción laboral
mycursor.execute('''CREATE TABLE IF NOT EXISTS satisfaction_scores (
                id_satisfaction INT AUTO_INCREMENT PRIMARY KEY,
                id_employee INT,
                environment_satisfaction INT,
                job_involvement INT,
                job_satisfaction INT,
                relationship_satisfaction INT,
                work_life_balance INT,
                performance_rating INT,
                FOREIGN KEY (id_employee) REFERENCES personal_info(id_employee))''')

# Tabla con los datos economicos y de las politicas de participación
mycursor.execute('''CREATE TABLE IF NOT EXISTS economic_info (
                id_economic INT AUTO_INCREMENT PRIMARY KEY,
                id_employee INT,
                monthly_income FLOAT,
                percent_salary_hike INT,
                stock_option_level INT,
                FOREIGN KEY (id_employee) REFERENCES personal_info(id_employee))''')

# Tabla con los datos de la trayectoria laboral de los empleados
mycursor.execute('''CREATE TABLE IF NOT EXISTS career_progression (
                id_career INT AUTO_INCREMENT PRIMARY KEY,
                id_employee INT,
                attrition VARCHAR(5), 
                num_companies_worked INT,
                total_working_years INT,
                training_times_last_year INT,
                years_at_company INT,
                years_since_last_promotion INT,
                years_with_curr_manager INT,
                FOREIGN KEY (id_employee) REFERENCES personal_info(id_employee))''')

#confirmacion de los cambios
cnx.commit()

print("Tablas creadas correctamente: Expelliarmus")

## 2. Insercción de datos en las tablas

In [ ]:
# Cargar el dataset
df_talento = pd.read_csv('df_talento_limpio.csv').fillna("")
# Creamos la columna vacía para que la inserción no falle al insertar datos de una columna que no existia
df_talento['name'] = None  # 


# INSERTAR EN personal_info Y MAPEAR IDs
query_personal = """INSERT INTO personal_info (employee_number, name, age, gender, marital_status, distance_from_home, date_birth, remote_work, education, education_field)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

personal_data = df_talento[['employee_number', 'name', 'age', 'gender', 'marital_status', 'distance_from_home', 'date_birth', 'remote_work', 'education', 'education_field']].values.tolist()

mycursor.executemany(query_personal, personal_data)

cnx.commit()
print("Datos guardados en personal_info. Travesura realizada.")


# MAPEAR id_employee generado
mycursor.execute("SELECT id_employee, employee_number FROM personal_info")
id_mapping = dict(mycursor.fetchall())

# Creamos la nueva columna en df_talento
df_talento['id_employee'] = df_talento['employee_number'].map(id_mapping)
if df_talento['id_employee'].isnull().any():
    print("Atención: Hay empleados sin id_employee mapeado. ¡Vigilancia constante!")


# INSERTAR EN employment_info
query_employment = """INSERT INTO employment_info (id_employee, department, job_role, job_level, business_travel, overtime)
                     VALUES (%s, %s, %s, %s, %s, %s)"""
mycursor.executemany(query_employment, df_talento[['id_employee', 'department', 'job_role', 'job_level', 'business_travel', 'overtime']].values.tolist())
print("Datos registrados en employment_info. Travesura realizada.")


# INSERTAR EN satisfaction_scores
query_satisfaction = """INSERT INTO satisfaction_scores (id_employee, environment_satisfaction, job_involvement, job_satisfaction, relationship_satisfaction, work_life_balance, performance_rating)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)"""
mycursor.executemany(query_satisfaction, df_talento[['id_employee', 'environment_satisfaction', 'job_involvement', 'job_satisfaction', 'relationship_satisfaction', 'work_life_balance', 'performance_rating']].values.tolist())
print("Datos ejecutados en satisfaction_scores. Travesura realizada.")


# INSERTAR EN economic_info
query_economic = """INSERT INTO economic_info (id_employee, monthly_income, percent_salary_hike, stock_option_level) VALUES (%s, %s, %s, %s)"""
mycursor.executemany(query_economic, df_talento[['id_employee', 'monthly_income', 'percent_salary_hike', 'stock_option_level']].values.tolist())
print("Datos insertados en economic_info. Travesura realizada")


# INSERTAR EN career_progression
query_career = """INSERT INTO career_progression (id_employee, attrition, num_companies_worked, total_working_years, training_times_last_year, years_at_company, years_since_last_promotion, years_with_curr_manager)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
mycursor.executemany(query_career, df_talento[['id_employee', 'attrition', 'num_companies_worked', 'total_working_years', 'training_times_last_year', 'years_at_company', 'years_since_last_promotion', 'years_with_curr_manager']].values.tolist())
print("Datos cargados en career_progresion. Travesura realizada.")


# Commit final y cierre
cnx.commit()
print("Carga completada: Adentremonos en la noche y sigamos esa tentadora aventura.")

mycursor.close()
cnx.close()
print("Finite Incantatem.")
